# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler, also compare against the data from CMAPSAuxFunctions

First we import the necessary packages and create the global variables.

In [1]:
import numpy as np
import math
from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

#import CMAPSAuxFunctions

from old.tunableModel import TunableModel as TunableModel

Using TensorFlow backend.


TabError: inconsistent use of tabs and spaces in indentation (tunableModel.py, line 29)

# Define different types of Keras models

In [2]:
lambda_regularization = 0.20

def RNNModel(input_shape):
    """Define the RNN model"""
    
    #Create a sequential model
    model = Sequential()
    #model.add(Masking(mask_value=0, imput))
    model.add(LSTM(input_shape=input_shape))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model
    

def RULmodel_SN_6(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

# Default settings for the Data

In [3]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

window_size = 30
window_stride = 1
max_rul = 125

# Create the Data Handler

In [4]:
dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)
#dHandler_cmaps.load_data()

# Create Tunable Model using CMAPSAuxFunctions

In [5]:
nFeatures = len(selected_features)
shapeSN = nFeatures*window_size
modelRULSN = RULmodel_SN_6(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

tModel = TunableModel('ModelRUL_SN_1', modelRULSN, selected_features, data_folder, 'keras', window_size=window_size,
                      scaler = min_max_scaler, window_stride=window_stride, constRul = max_rul)

# Load data using both approaches

In [6]:
print("Loading with data handler")
dHandler_cmaps.load_data()
dHandler_cmaps.print_data()

print("Loading with aux functions")
tModel.loadData()
tModel.printData()

Loading with data handler
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(17731, 420)
(17731, 1)
Testing data (X, y)
(100, 420)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[ 641.82   1589.7    1400.6    ...  390.       39.05     23.411 ]
 [ 642.15   1591.82   1403.14   ...  392.       38.94     23.3353]
 [ 642.35   1587.99   1404.2    ...  392.       39.02     23.4999]
 [ 642.35   1582.79   1401.87   ...  392.       38.83     23.3506]
 [ 642.37   1582.85   1406.22   ...  392.       38.81     23.3092]]
[[125.]
 [125.]
 [125.]
 [125.]
 [125.]]
Testing data (X, y)
[[ 641.71   1588.45   1395.42   ...  393.       38.81     23.3552]
 [ 642.72   1589.15   1400.69   ...  391.       38.81     23.2618]
 [ 642.86   1588.9    1408.72   ...  395.       38.93     23.274 ]
 [ 642.45   1593.88   1407.84   ...  395.       38.58     23.2581]
 [ 642.58   1595.41   1408.83   ...  394.       38.75     23.4117]]
[[112.]
 [ 98.]
 [ 69.]
 [ 82.]
 [ 91.]]
Lo

AttributeError: module 'CMAPSAuxFunctions' has no attribute 'load_into_df'

# Create Tunable Model

In [4]:
#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]


#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*window_size
modelRULSN = RULmodel_SN_6(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

#min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
#min_max_scaler = None

tModel = SequenceTunableModelRegression('ModelRUL_SN_1', modelRULSN, lib_type='keras', data_handler=dHandler_cmaps)

# Load Data

In [6]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
tModel.data_scaler = min_max_scaler
#tModel.data_handler.change_dataset(2)
tModel.data_handler.sequence_length = 30
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 128

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
tModel.print_data()

Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 128. Cros-Validation ratio 0
Loading data from memory and recomputing dataframes
Printing shapes

Training data (X, y)
(17731, 420)
(17731,)
Testing data (X, y)
(100, 420)
(100,)
Printing first 5 elements

Training data (X, y)
[[-0.58075601 -0.0455243  -0.27982732 ... -0.81818182  0.43307087
   0.4679733 ]
 [-0.35395189  0.0629156  -0.18014129 ... -0.45454545  0.25984252
   0.25294702]
 [-0.21649485 -0.13299233 -0.13854003 ... -0.45454545  0.38582677
   0.72049425]
 [-0.21649485 -0.39897698 -0.2299843  ... -0.45454545  0.08661417
   0.29640676]
 [-0.20274914 -0.39590793 -0.05926217 ... -0.45454545  0.05511811
   0.17880983]]
[128. 128. 128. 128. 128.]
Testing data (X, y)
[[-0.65635739 -0.10946292 -0.48312402 ... -0.27272727  0.05511811
   0.30947309]
 [ 0.03780069 -0.07365729 -0.27629513 ... -0.63636364  0.05511811
   0.04416986]
 [ 0.13402062 -0.08644501  0.038854   ...  0.09090909  0.24409449
   0.07882403]


# Train the model and test some Tunable Model functionalities

In [7]:
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
tModel.epochs = 100
tModel.train_model(learningRate_scheduler=lrate, verbose=1)

NameError: name 'CMAPSAuxFunctions' is not defined

In [8]:
tModel.evaluate_model(['rhs', 'rmse'], round=2, setLimits=[7,145])
print("scores")

cScores = tModel.scores
rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time
iterations = 10

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, tModel.y_predicted, tModel.y_predicted_rounded, tModel.y_test):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 132us/step
scores
Engine 1, Predicted RUL [ 113.59394073], Rounded RUL 113.0, Real RUL 112.0
Engine 2, Predicted RUL [ 115.27581024], Rounded RUL 115.0, Real RUL 98.0
Engine 3, Predicted RUL [ 64.98041534], Rounded RUL 64.0, Real RUL 69.0
Engine 4, Predicted RUL [ 86.50939941], Rounded RUL 86.0, Real RUL 82.0
Engine 5, Predicted RUL [ 98.51219177], Rounded RUL 98.0, Real RUL 91.0
Engine 6, Predicted RUL [ 102.53627777], Rounded RUL 102.0, Real RUL 93.0
Engine 7, Predicted RUL [ 106.88496399], Rounded RUL 106.0, Real RUL 91.0
Engine 8, Predicted RUL [ 89.14979553], Rounded RUL 89.0, Real RUL 95.0
Engine 9, Predicted RUL [ 115.53969574], Rounded RUL 115.0, Real RUL 111.0
Engine 10, Predicted RUL [ 89.71224213], Rounded RUL 89.0, Real RUL 96.0
Engine 11, Predicted RUL [ 93.70278168], Rounded RUL 93.0, Real RUL 97.0
Engine 12, Predicted RUL [ 102.93409729], Rounded RUL 102.0, Real RUL 124.0
Engine 13, Predicted RUL [ 87.24149323], Rounded RUL 8